In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import os

# 1. Đường dẫn file
QT_FILE = '/content/drive/MyDrive/DATA/SAVE_MODEL/MODEL_SCORE_PREDICTION/QUA_TRINH/model1/272128_QT_prediction.csv'
TH_FILE = '/content/drive/MyDrive/DATA/SAVE_MODEL/MODEL_SCORE_PREDICTION/THUC_HANH/model1/271797_TH_prediction.csv'
CK_FILE = '/content/drive/MyDrive/DATA/SAVE_MODEL/MODEL_SCORE_PREDICTION/CUOI_KY/model1/272131_CK_prediction.csv'

OUTPUT_DIR = '/content/drive/MyDrive/DATA/SAVE_MODEL/MODEL_SCORE_PREDICTION/TBTL/Model1'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 2. Trọng số điểm
TRONG_SO_QT = 0.20
TRONG_SO_TH = 0.40
TRONG_SO_CK = 0.40

print(f"Trọng số: QT={TRONG_SO_QT*100:.0f}%, TH={TRONG_SO_TH*100:.0f}%, CK={TRONG_SO_CK*100:.0f}%")

# 3. Đọc và xử lý file
try:
    # Đọc file QT
    qt_df = pd.read_csv(QT_FILE)
    qt_score_col = next((col for col in qt_df.columns if 'pred' in col.lower() or 'qt' in col.lower()), qt_df.columns[1])
    qt_data = qt_df[['hash', qt_score_col]].rename(columns={qt_score_col: 'QT'})

    # Đọc file TH
    th_df = pd.read_csv(TH_FILE)
    th_score_col = next((col for col in th_df.columns if 'pred' in col.lower() or 'th' in col.lower()), th_df.columns[1])
    th_data = th_df[['hash', th_score_col]].rename(columns={th_score_col: 'TH'})

    # Đọc file CK
    ck_df = pd.read_csv(CK_FILE)
    ck_score_col = next((col for col in ck_df.columns if 'pred' in col.lower() or 'ck' in col.lower()), ck_df.columns[1])
    ck_data = ck_df[['hash', ck_score_col]].rename(columns={ck_score_col: 'CK'})

    print(f"Đọc thành công: QT({len(qt_data)}), TH({len(th_data)}), CK({len(ck_data)})")

except Exception as e:
    print(f"Lỗi đọc file: {e}")
    exit()

# 4. Gộp dữ liệu và tính TBTL
result = qt_data.merge(th_data, on='hash', how='inner')  # Sử dụng inner join để đảm bảo có đủ cả 3 điểm
result = result.merge(ck_data, on='hash', how='inner')

# Tính điểm TBTL
result['TBTL'] = (
    result['QT'] * TRONG_SO_QT +
    result['TH'] * TRONG_SO_TH +
    result['CK'] * TRONG_SO_CK
).round(6)

print(f"Tính toán xong cho {len(result)} học sinh")

# 5. Lưu kết quả
final_result = result[['hash', 'TBTL']].copy()
filepath = os.path.join(OUTPUT_DIR, "TBTL_prediction.csv")
final_result.to_csv(filepath, index=False)

print(f"Đã lưu: {filepath}")

# 6. Hiển thị kết quả
print(f"\nMẫu kết quả (5 dòng đầu):")
print(final_result.head())

print(f"\nĐiểm TBTL từ {final_result['TBTL'].min():.2f} đến {final_result['TBTL'].max():.2f}")

# 7. Tải file (nếu trong Colab)
try:
    from google.colab import files
    files.download(filepath)
    print("Đã tải file thành công!")
except:
    print("Không thể tự động tải file (có thể không phải Colab)")

print(f"\nHOÀN THÀNH! File có {len(final_result)} dòng dữ liệu.")

Trọng số: QT=20%, TH=40%, CK=40%
Đọc thành công: QT(1489), TH(1489), CK(1489)
Tính toán xong cho 1489 học sinh
Đã lưu: /content/drive/MyDrive/DATA/SAVE_MODEL/MODEL_SCORE_PREDICTION/TBTL/Model1/TBTL_prediction.csv

Mẫu kết quả (5 dòng đầu):
                                       hash   TBTL
0  00b6dd4fc7eb817e03708c532016ef30ce564a61  5.966
1  00bef8afee8f3c595d535c9c03c490cac1a4f021  7.312
2  01122b3ef7e59b84189e65985305f575d6bdf83c  6.680
3  0134f9f410c65ad0e8c2254a7e9288670e02a183  5.408
4  013de369c439ab0ead8aa7da64423aa395a8be39  6.968

Điểm TBTL từ 3.08 đến 9.04


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Đã tải file thành công!

HOÀN THÀNH! File có 1489 dòng dữ liệu.
